3 Tables:
players: catalog of many qbs, rbs, wrs, and tes
player_stats: yearly statistics for every player
draft_board: overall and positional draft rankings from 2011-present

In [36]:
#Import dependencies
import nfl_data_py as nfl
import pandas as pd
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import getpass
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [16]:
#Create SQL Engine
DB_NAME = "NFLDB_1"
USERNAME = "root"
PASSWORD = getpass.getpass("Enter your MySQL password: ")
HOST = "localhost"

engine = create_engine(f"mysql+pymysql://{USERNAME}:{PASSWORD}@{HOST}/{DB_NAME}")

In [21]:
#Create players table
players = nfl.import_players()
players = players[['gsis_id', 'draft_number', 'football_name', 'last_name', 'position', 'rookie_year', 'height', 'weight']]
players["draft_number"] = players["draft_number"].astype("object")
players.loc[:, "draft_number"] = players["draft_number"].fillna("UDFA")
players = players.dropna()
players = players[players["position"].isin(['QB', 'RB', 'WR', 'TE'])]

players['football_name'] = players['football_name'].astype('string')
players['last_name'] = players['last_name'].astype('string')
players['draft_number'] = players['draft_number'].astype('string')
players['position'] = players['position'].astype('string')
players['rookie_year'] = pd.to_numeric(players['rookie_year'], errors='coerce').astype('Int64')
players['height'] = pd.to_numeric(players['height'], errors='coerce').astype('Int64')
players['weight'] = pd.to_numeric(players['weight'], errors='coerce').astype('Int64')
players = players.rename(columns={'football_name': 'first_name', 'gsis_id': 'id'})

players.to_sql('players', con=engine, if_exists='append', index=False)

2651

In [22]:
#Create player stats table
player_stats = nfl.import_seasonal_data(range(2014, 2025), s_type="REG")

#Stats omitted: season_type, pacr, dakota, racr, wopr_x, wopr_y
player_stats = player_stats[['player_id', 'season', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'target_share', 'air_yards_share', 
       'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',
       'games', 'tgt_sh', 'ay_sh', 'yac_sh', 'ry_sh', 'rtd_sh',
       'rfd_sh', 'rtdfd_sh', 'dom', 'w8dom', 'yptmpa', 'ppr_sh']]

player_stats.to_sql('player_stats', engine, if_exists='append', index=False)

6682

In [50]:
def get_best_match(player_name, player_id_mapping, threshold=85):
    #85 threshold is the sweetspot
    # Check for exact match first
    exact_match = player_id_mapping[player_id_mapping['player_name'] == player_name]
    if not exact_match.empty:
        return exact_match['id'].values[0]
    
    # If no exact match, use fuzzywuzzy to find the best match
    match = process.extractOne(player_name, player_id_mapping['player_name'], scorer=fuzz.partial_ratio)
    
    if match and match[1] >= threshold:
        # If the match score is above the threshold, return the corresponding player_id
        best_match_player_name = match[0]
        player_id = player_id_mapping.loc[player_id_mapping['player_name'] == best_match_player_name, 'id'].values[0]
        return player_id
    else:
        # If no match or score is below threshold, return None or handle it as needed
        return None

In [54]:
#Create draft board table
df = pd.read_csv('adp-every-season.csv')
df = df[df['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]
ovr_rank = qb_rank = rb_rank = wr_rank = te_rank = 1
current_year = 2011 #May have to change based on csv
for index, row in df.iterrows():
    if row['Season'] > current_year:
        ovr_rank = qb_rank = rb_rank = wr_rank = te_rank = 1
        current_year = row['Season']
    df.loc[index, 'overall_rank'] = ovr_rank
    ovr_rank+=1
    if row['Pos'] == 'QB':
        df.loc[index, 'pos_rank'] = qb_rank
        qb_rank+=1
    elif row['Pos'] == 'RB':
        df.loc[index, 'pos_rank'] = rb_rank
        rb_rank+=1
    elif row['Pos'] == 'WR':
        df.loc[index, 'pos_rank'] = wr_rank
        wr_rank+=1    
    else :
        df.loc[index, 'pos_rank'] = te_rank
        te_rank+=1
df = df.drop(columns='Overall')
df.columns = ['player_name', 'pos', 'team', 'season', 'overall_rank', 'pos_rank']
df = df[df['player_name'].str.split().str.len() > 1]

with engine.connect() as connection:
    query = """SELECT id, CONCAT(first_name, ' ', last_name) AS player_name FROM players"""
    player_id_mapping = pd.read_sql(query, connection)
    
merged_df = df.copy(deep=True)
merged_df['player_id'] = merged_df['player_name'].apply(lambda name: get_best_match(name, player_id_mapping))
merged_df = merged_df.dropna(subset=['player_id'])
merged_df[['player_id', 'player_name', 'pos', 'team', 'season', 'overall_rank', 'pos_rank']].to_sql(
    'draft_board', engine, if_exists='append', index=False
)

2207